In [ ]:

from math import exp,sqrt
import numpy as np
import pdb
import matplotlib.pyplot as plt

class geometric_brownian_motion_simulation:
    '''
        Parameters
    ----------
    initial price : `float`
        The initial price of a stock at time 0.
    drift_term : `float`
        The .
    sigma : `float`
        The 'volatility' of the asset.
    increments : `float`
        The parameter used to govern the Pareto distribution
        shape for the generation of volume data.
    '''


    def __init__(self, initial_price, drift_term, sigma, increments, time_interval, n_simulations):
        self.initial_price = initial_price
        self.drift_term = drift_term 
        self.sigma = sigma 
        self.increments = increments 
        self.time_interval = time_interval
        self.simulations = n_simulations

    
    def calculate_gbm():
        return

    def run_simulations(self, n : int ):
        '''
        Determine how many brownian motion simulations you would like to generate
        
        Returns
        --------
        'dict' 
            A dictionary in which the key, value pairs are the index of each simulation and a list of the generated outcomes
        
        '''
        results = {}
        for i in n:
            results[i] = self.calculate_gbm()
        return results

    def plot_simulations(self):
        '''
        Plot all simulations once they are ready
        '''

        return


    def __call__(self):
        '''
        Call method to generate and plot all simulations after all variables are created
        '''
        simulations = self.run_simulations(self.simulations)
        self.plot_simulations(simulations)
        return


